In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np
import matplotlib.pyplot as plt

In [3]:
lstm = nn.LSTM(input_size=3,
       hidden_size=5,
       num_layers=1)

In [ ]:
# batch_size : 2
# sequence length : 10
# input dim : 3
# hidden dim : 5

In [6]:
t_in = torch.randn([10,2,3])
h_in = torch.randn([1,2,5])
c_in = torch.randn([1,2,5])

In [8]:
t_out = lstm(t_in, (h_in, c_in))

In [12]:
output = t_out[0]
h_out = t_out[1][0]
c_out = t_out[1][1]

In [13]:
output.shape, h_out.shape, c_out.shape

(torch.Size([10, 2, 5]), torch.Size([1, 2, 5]), torch.Size([1, 2, 5]))

In [14]:
i0 = torch.randn([6,3])
i1 = torch.randn([7,3])
i2 = torch.randn([8,3])

In [35]:
batch = [i0, i1, i2]

In [36]:
length_info = [len(i) for i in batch]

In [37]:
length_info

[6, 7, 8]

In [38]:
max_length = max(length_info)

In [45]:
batch_size = len(batch)

In [56]:
mask =torch.arange(max_length).expand([batch_size, max_length]) < torch.LongTensor(np.array(length_info).reshape(3,1))

In [53]:
k

tensor([[ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True]])

In [15]:
vt_in = nn.utils.rnn.pad_sequence([i0,i1,i2])

In [16]:
vt_in.shape

torch.Size([8, 3, 3])

In [19]:
vh_in = torch.randn([1,3,5])
vc_in = torch.randn([1,3,5])

In [21]:
vt_out = lstm(vt_in, (vh_in, vc_in))

In [22]:
voutput = vt_out[0]
vh_out = vt_out[1][0]
vc_out = vt_out[1][1]

In [23]:
voutput.shape, vh_out.shape, vc_out.shape

(torch.Size([8, 3, 5]), torch.Size([1, 3, 5]), torch.Size([1, 3, 5]))

In [65]:
mask = mask.permute([1,0])

In [70]:
mask = mask.unsqueeze(-1)

In [71]:
mask = mask.expand_as(voutput)

In [66]:
mask.shape

torch.Size([8, 3])

In [58]:
voutput.shape

torch.Size([8, 3, 5])

AttributeError: module 'torch' has no attribute 'inf'

In [75]:
moutput = voutput * mask.float()

In [77]:
moutput

tensor([[[-0.0539, -0.3788,  0.0852, -0.1536, -0.2886],
         [-0.0043, -0.0285,  0.3031, -0.0230,  0.1119],
         [-0.1541,  0.1305,  0.1078,  0.1657,  0.2549]],

        [[-0.0347, -0.3870,  0.0985, -0.1134, -0.0167],
         [-0.1134, -0.1455,  0.1411, -0.0110, -0.0616],
         [-0.1460,  0.0687,  0.0940,  0.1692,  0.0506]],

        [[-0.0252, -0.3246,  0.1148, -0.1033, -0.0540],
         [-0.1322, -0.3093, -0.0144, -0.0122,  0.1745],
         [-0.1416,  0.1127,  0.1463,  0.0897, -0.1610]],

        [[-0.0676, -0.3252,  0.1070, -0.0736, -0.0243],
         [-0.1721, -0.3834,  0.0310, -0.0194, -0.0754],
         [ 0.1712,  0.3740,  0.3058, -0.0015, -0.1596]],

        [[-0.1469, -0.2941,  0.0923, -0.0516, -0.1520],
         [-0.1413, -0.3904,  0.0424, -0.0524, -0.0375],
         [-0.0707, -0.0736,  0.1367,  0.0206, -0.2370]],

        [[-0.0669, -0.3834,  0.0981, -0.0957,  0.0459],
         [-0.1690, -0.3249,  0.0419, -0.0370, -0.1685],
         [-0.1219, -0.2216,  0.0983,  

In [79]:
ta = np.array([
    [False , False, True, False, False],
    [False, False, False, False, False],
    [True, False, False, False, True]
]
)

In [96]:
np.cumprod(~ta, axis=1, dtype=np.bool)

array([[ True,  True, False, False, False],
       [ True,  True,  True,  True,  True],
       [False, False, False, False, False]])

In [92]:
torch.BoolTensor(~ta)

tensor([[ True,  True, False,  True,  True],
        [ True,  True,  True,  True,  True],
        [False,  True,  True,  True, False]])

In [89]:
np.argwhere(ta)

array([[0, 2],
       [2, 0],
       [2, 4]])

In [85]:
list(zip(*np.where(ta)))

[(0, 2), (2, 0), (2, 4)]